In [32]:
# initialize
from tqdm import tqdm
from time import sleep

import glob
import parselmouth
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

!ls {patho_path}

functional      hyperfunctional organic         psychogenic


In [33]:
# Process wav files to get Jitter, Shimmer, HNR, and MFCC

def get_voice_data(_path):
    # select .wav files only
    wav_files = glob.glob(_path + "/*.wav")

    n_list = []
    tone_list = []
    syllab_list = []

    j_list = []
    s_list = []
    h_list = []

    # for wav_file in wav_files:
    for wav_file in tqdm(wav_files): # tqdm shows the progress bar
        sound = parselmouth.Sound(wav_file) # sound object from wav file
        pitch = sound.to_pitch()
        pulses = parselmouth.praat.call([sound, pitch], "To PointProcess (cc)")

        # name analysis
        name = os.path.basename(wav_file).split(".")[0]  
        
        ## tone
        if "l" in name:
            tone_list.append("l")
        elif "n" in name:
            tone_list.append("n")
        elif "h" in name:
            tone_list.append("h")

        ## syllable
        if "a" in name:
            syllab_list.append("a")
        elif "i" in name:
            syllab_list.append("i")
        elif "u" in name:
            syllab_list.append("u")
        # jitter
        jitter_local = parselmouth.praat.call(pulses, "Get jitter (local)", 0.0, 0.0, 0.0001, 0.02, 1.3) * 100

        # shimmer
        shimmer_local = parselmouth.praat.call([sound, pulses], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

        # HNR
        harmonicity = parselmouth.praat.call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        hnr = parselmouth.praat.call(harmonicity, "Get mean", 0, 0)
        
        # Append to numpy array
        n_list.append(name)
        j_list.append(jitter_local)
        s_list.append(shimmer_local)
        h_list.append(hnr)

        # MFCC
        mfcc_object = sound.to_mfcc(number_of_coefficients=13)
        mfcc_arr = mfcc_object.to_array()
        mfcc_dic = {}
        for i in range(1,len(mfcc_arr)):
            mfcc_dic["MFCC-"+str(i)] = mfcc_arr[i]
        mfcc_df = pd.DataFrame.from_dict(mfcc_dic)

    # create dataframe
    df = pd.DataFrame({"Name":pd.Series(n_list),
                        "Type": np.nan,
                        "Tone": pd.Series(tone_list),
                        "Syllab": pd.Series(syllab_list),
                           "Jitter":pd.Series(j_list),
                           "Shimmer":pd.Series(s_list),
                           "HNR":pd.Series(h_list)})
    df["Type"]= _path.split("/")[-1] # identify type: my_data, healthy, functional etc...
    new_df = pd.concat([df, mfcc_df], axis=1, sort=False)
    new_df = new_df.dropna() # some data are missing jitter, shimmer, hnr for some reason it seems..?
    return new_df


In [34]:
def generate_jshmfcc(dataset_type, dataset_path):
    healthy_df = get_voice_data(dataset_path + "/healthy")
    functional_df = get_voice_data(dataset_path + "/pathological/functional")
    hyperfunctional_df = get_voice_data(dataset_path + "/pathological/hyperfunctional")
    organic_df = get_voice_data(dataset_path + "/pathological/organic")
    psychogenic_df = get_voice_data(dataset_path + "/pathological/psychogenic")

    # Combine the results into one dataframe
    frames = [healthy_df, functional_df, hyperfunctional_df, organic_df, psychogenic_df]
    combined_df = pd.concat(frames)
    combined_df = combined_df.dropna()
    return combined_df


In [35]:
# filepath for the test and train datasets
test_path = "/Users/leochoo/dev/VoiceDisorderSVM/data/SVD/test_audio"
train_path = "/Users/leochoo/dev/VoiceDisorderSVM/data/SVD/train_audio"

In [36]:
# generate voice report for test dataset
test_report = generate_jshmfcc("test", test_path)
test_report

100%|██████████| 27/27 [00:02<00:00,  9.88it/s]


,Name,Type,Tone,Syllab,Jitter,Shimmer,HNR,MFCC-1,MFCC-2,MFCC-3,MFCC-4,MFCC-5,MFCC-6,MFCC-7,MFCC-8,MFCC-9,MFCC-10,MFCC-11,MFCC-12,MFCC-13
0,1-i_l,healthy,l,i,0.238779,0.019045,19.410768,401.683204,-106.091207,-16.204958,-177.451287,-80.432412,-167.325435,-70.950945,60.705756,-18.078235,-92.995773,-13.111788,-4.964332,-29.427566
1,2-u_h,healthy,h,u,0.349111,0.023441,28.655604,405.945214,-107.319951,-6.467390,-168.654549,-83.775023,-164.000390,-57.471754,48.452583,-6.290534,-82.880371,-21.278743,10.481437,-38.131368
2,1-i_n,healthy,n,i,0.209544,0.007423,26.996682,412.992202,-105.668520,4.367001,-173.748357,-77.660894,-151.323178,-68.017679,71.309964,-10.580220,-88.565720,-7.091164,3.198551,-33.208541
3,2-u_l,healthy,l,u,1.069854,0.041115,25.108378,414.991160,-97.516727,14.790359,-171.141081,-67.202782,-142.208466,-72.015559,82.295822,-4.728251,-89.755092,6.052225,-1.277937,-27.437695
4,2-u_n,healthy,n,u,0.413457,0.031538,24.573556,406.918197,-86.251020,10.266493,-182.298364,-60.985385,-148.072712,-66.949520,75.605251,-3.342546,-89.780277,-3.320846,8.403894,-32.433375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,151-a_l,psychogenic,l,a,0.100698,0.010602,32.236174,542.278005,-8.265111,-30.852985,-106.179846,-30.792265,-22.565280,-42.890834,29.361151,26.303133,-40.181743,-47.681269,38.188908,-16.714789
23,366-u_n,psychogenic,n,u,0.201356,0.015767,31.531414,536.969358,-3.689975,-21.333714,-120.413892,-17.823602,-23.398328,-48.960628,31.284078,32.925703,-45.128236,-41.538127,34.274904,-24.623071
24,741-a_h,psychogenic,h,a,0.281613,0.026882,28.802297,536.136757,-8.291563,-18.436971,-128.908856,-12.679275,-27.813144,-46.305927,37.134118,20.319001,-47.234094,-41.778117,34.873738,-26.046758
25,366-u_l,psychogenic,l,u,0.389981,0.039966,23.895287,543.660395,0.800456,-29.287699,-121.155722,-10.923893,-32.041889,-46.673164,35.578120,20.406814,-39.499129,-40.591833,23.920612,-20.058146


In [37]:
# generate voice report for train dataset
train_report = generate_jshmfcc("train", train_path)
train_report

100%|██████████| 792/792 [01:58<00:00,  6.71it/s]


,Name,Type,Tone,Syllab,Jitter,Shimmer,HNR,MFCC-1,MFCC-2,MFCC-3,MFCC-4,MFCC-5,MFCC-6,MFCC-7,MFCC-8,MFCC-9,MFCC-10,MFCC-11,MFCC-12,MFCC-13
0,743-u_n,healthy,n,u,1.164692,0.042058,24.772758,517.867690,157.285453,-67.337663,-75.557579,-47.383544,-35.418313,28.738609,29.240641,-3.077040,-8.491168,-37.546175,16.018237,-21.418999
1,56-i_n,healthy,n,i,0.155393,0.011061,26.922863,507.653534,146.597634,-51.805445,-67.727368,-51.745891,-42.226560,39.406267,17.952366,-4.490649,-5.343768,-46.048275,34.265482,-21.777943
2,39-a_l,healthy,l,a,0.224576,0.027684,27.151732,519.698504,138.201660,-75.513161,-55.364543,-29.771235,-46.565257,28.243726,11.190055,-3.965739,11.588003,-44.706164,22.710550,-18.795814
3,29-a_l,healthy,l,a,0.498826,0.038859,20.022510,512.159853,128.304605,-53.085907,-59.688275,-45.695365,-33.831168,15.513842,17.563326,3.432073,8.504941,-49.367170,21.904375,-13.309917
4,46-i_n,healthy,n,i,0.532008,0.023150,22.705093,520.859534,137.339353,-66.339532,-72.067937,-37.852838,-29.106708,16.681151,18.563943,7.352762,7.298131,-52.548622,26.928928,-22.352170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2213-u_n,psychogenic,n,u,0.705847,0.031420,31.770763,430.996203,22.751481,165.731823,115.509842,117.175360,32.165182,-86.782371,-109.202137,6.666115,17.104409,-18.310357,-30.411579,-29.861823
222,1384-a_h,psychogenic,h,a,0.391448,0.030339,22.419719,415.754965,58.695049,162.040128,117.238511,120.276357,30.181962,-72.120151,-87.305744,21.471254,25.510606,-13.336603,-31.576838,-24.068587
223,2526-u_h,psychogenic,h,u,1.114191,0.066075,19.596194,416.033054,24.016344,163.129059,114.941535,131.034301,20.795789,-78.938331,-108.746561,4.004710,18.889512,-18.150885,-32.550457,-29.587528
224,2107-u_n,psychogenic,n,u,0.724640,0.021346,28.980100,411.106020,35.644724,165.359706,111.082177,129.588425,33.879828,-75.525073,-78.843720,17.038367,27.385695,-28.341914,-9.171042,-22.513063


In [38]:
# Save the outputs to the processed data directory
test_report.to_csv ("./data/processed/test_SVD_j_s_hnr_mfcc.csv", index = False, header=True)
print("Test data exported")
train_report.to_csv ("./data/processed/train_SVD_j_s_hnr_mfcc.csv", index = False, header=True)
print("Train data exported")



Test data exported
Train data exported
